In [ ]:
# ============================================================================
# IMPORTS AND SETUP
# ============================================================================
import sys
from pathlib import Path
from typing import Tuple, Optional, Union

import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, FancyArrowPatch, Ellipse
from matplotlib.collections import LineCollection
import scipy.signal
from scipy.signal import hilbert, correlate
from scipy.stats import circmean, circstd

# Add src to path for local imports
sys.path.insert(0, str(Path.cwd().parents[2]))

from src.colors import COLORS
from src.phase import compute_phase_difference, compute_plv_simple

# Color aliases
PRIMARY_BLUE = COLORS["signal_1"]      # Sky Blue
PRIMARY_RED = COLORS["signal_2"]       # Rose Pink
PRIMARY_GREEN = COLORS["signal_3"]     # Sage Green
SECONDARY_ORANGE = COLORS["signal_4"]  # Golden
SECONDARY_PURPLE = COLORS["high_sync"] # Purple
SUBJECT_1 = COLORS["signal_1"]         # For hyperscanning
SUBJECT_2 = COLORS["signal_2"]         # For hyperscanning

# Sampling frequency
fs = 256  # Hz

# Random seed for reproducibility
np.random.seed(42)

print("✓ Imports successful!")
print(f"NumPy version: {np.__version__}")

---

## 1. Introduction — The Elephant in the Room 🐘

Before we learn **any** connectivity metric, we must confront the central problem of EEG connectivity analysis: **volume conduction**.

EEG measures electrical activity at the scalp surface. But between the neurons generating this activity and our electrodes lie multiple layers of tissue: brain matter, cerebrospinal fluid (CSF), skull, and scalp. All these tissues conduct electricity.

**The fundamental problem**: A single electrical source in the brain doesn't stay localized — its electrical field spreads through the conductive tissues and reaches **multiple electrodes simultaneously**. This means:

- If two electrodes both "see" the same brain source...
- Their signals will be correlated...
- They will appear to be "connected"...
- **But this is NOT real neural connectivity!**

This artifact — called **spurious connectivity** — is the single biggest threat to the validity of EEG connectivity studies. Papers have been published, conclusions drawn, and theories built on connectivity patterns that were largely or entirely due to volume conduction.

> ⚠️ **Critical Warning**: If you don't understand volume conduction, your connectivity results may be **meaningless**.

This problem is even MORE critical in hyperscanning research, where we want to distinguish true inter-brain coupling from artifacts. The good news: we have solutions. But first, let's deeply understand the problem.

---

## 2. What is Volume Conduction?

### The Physics (Simplified)

Brain activity consists of electrical currents flowing through neurons. These currents create **electrical fields** that extend into the surrounding tissue. The key properties:

1. **Tissues are conductive**: Brain, CSF, skull, and scalp all conduct electricity (like a network of resistors)
2. **Fields spread spatially**: A single source creates a field that extends in all directions
3. **Multiple electrodes detect each source**: The field reaches many points on the scalp
4. **Signal at each electrode is a mixture**: Every electrode picks up contributions from MANY brain sources

### An Analogy

Imagine listening to an orchestra from **outside** the concert hall:
- All instruments are mixed together
- You can't isolate the violin from the cello
- The sound at different positions outside is similar (same sources, slight variations)

This is exactly what happens with EEG: we're "listening" from outside the skull, and all brain sources are mixed together at each electrode.

### Technical Term

This phenomenon is called **field spread** or **volume conduction** because the electrical fields are conducted through the volume of tissue between sources and sensors.

In [ ]:
# ============================================================================
# VISUALIZATION 1: Field Spread from Source to Electrodes
# ============================================================================

fig, ax = plt.subplots(figsize=(10, 8))

# Draw head outline (simplified as ellipse)
head = Ellipse((0.5, 0.4), 0.8, 0.7, fill=False, edgecolor='black', linewidth=2)
ax.add_patch(head)

# Draw brain region (inner ellipse)
brain = Ellipse((0.5, 0.35), 0.6, 0.45, fill=True, facecolor='#FFE4E1', 
                edgecolor='gray', linewidth=1, alpha=0.5)
ax.add_patch(brain)

# Source location (single dipole in brain)
source_x, source_y = 0.5, 0.35
ax.plot(source_x, source_y, 'o', markersize=15, color=PRIMARY_RED, 
        markeredgecolor='darkred', markeredgewidth=2, zorder=10)
ax.annotate('Brain Source', (source_x, source_y - 0.08), ha='center', 
            fontsize=11, fontweight='bold', color='darkred')

# Draw concentric circles representing field spread
for radius, alpha in [(0.1, 0.4), (0.2, 0.3), (0.3, 0.2), (0.4, 0.1)]:
    circle = Circle((source_x, source_y), radius, fill=False, 
                     edgecolor=PRIMARY_RED, linewidth=1.5, alpha=alpha, linestyle='--')
    ax.add_patch(circle)

# Electrode positions on scalp
electrode_positions = [
    (0.2, 0.65, 'E1'),
    (0.5, 0.75, 'E2'),
    (0.8, 0.65, 'E3'),
    (0.35, 0.55, 'E4'),
    (0.65, 0.55, 'E5'),
]

# Draw electrodes and lines from source
for ex, ey, label in electrode_positions:
    # Line from source to electrode (showing field reaching electrode)
    distance = np.sqrt((ex - source_x)**2 + (ey - source_y)**2)
    line_alpha = max(0.2, 1 - distance * 2)  # Closer = stronger
    ax.plot([source_x, ex], [source_y, ey], '-', color=PRIMARY_RED, 
            alpha=line_alpha, linewidth=2)
    
    # Electrode marker
    ax.plot(ex, ey, 's', markersize=12, color=PRIMARY_BLUE, 
            markeredgecolor='darkblue', markeredgewidth=2, zorder=10)
    ax.annotate(label, (ex, ey + 0.05), ha='center', fontsize=10, fontweight='bold')

# Annotations
ax.annotate('Electrical field\nspreads through tissue', (0.85, 0.35), 
            fontsize=10, ha='left', style='italic',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

ax.annotate('All electrodes\ndetect the SAME source!', (0.1, 0.2), 
            fontsize=11, ha='left', fontweight='bold', color='red',
            bbox=dict(boxstyle='round', facecolor='white', edgecolor='red'))

ax.set_xlim(-0.1, 1.1)
ax.set_ylim(0, 0.9)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Volume Conduction: One Source → Multiple Electrodes', 
             fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("📡 A single brain source contributes to signals at ALL nearby electrodes.")
print("   The closer the electrode, the stronger the contribution.")

---

## 3. The Problem for Connectivity

Now let's see why volume conduction is devastating for connectivity analysis.

### The Scenario

Imagine a **single** oscillating source in the brain (say, a 10 Hz alpha rhythm). Two nearby electrodes, A and B, both pick up this source:

- Signal at A = 0.8 × source + noise
- Signal at B = 0.5 × source + noise

### What Happens When We Measure Connectivity?

- **Correlation**: Very high! (Both signals contain the same source)
- **Phase synchronization (PLV)**: Nearly perfect! (Same oscillation = same phase)
- **Coherence**: Very high! (Same frequency content, locked phase)

### The Problem

We would conclude: "Strong connectivity between regions A and B!"

**But this is WRONG.** There is no connection between brain regions. There's just ONE source appearing at TWO electrodes. This is called **spurious connectivity** or **artificial coupling**.

In [ ]:
# ============================================================================
# VISUALIZATION 2: Volume Conduction Simulation
# ============================================================================

# Create a single source: 10 Hz oscillation
duration = 2.0
t = np.arange(0, duration, 1/fs)
n_samples = len(t)

# Source signal (alpha rhythm)
source = np.sin(2 * np.pi * 10 * t)

# Two electrodes receive weighted versions of the source + noise
noise_level = 0.15
electrode_A = 0.8 * source + noise_level * np.random.randn(n_samples)
electrode_B = 0.5 * source + noise_level * np.random.randn(n_samples)

# Plot
fig, axes = plt.subplots(3, 1, figsize=(12, 7), sharex=True)

# Source
axes[0].plot(t, source, color=PRIMARY_RED, linewidth=2)
axes[0].set_ylabel('Amplitude', fontsize=11)
axes[0].set_title('Brain Source (10 Hz oscillation)', fontsize=12, fontweight='bold')
axes[0].set_ylim([-1.5, 1.5])
axes[0].grid(True, alpha=0.3)

# Electrode A
axes[1].plot(t, electrode_A, color=PRIMARY_BLUE, linewidth=1.5)
axes[1].set_ylabel('Amplitude', fontsize=11)
axes[1].set_title('Electrode A (0.8 × source + noise)', fontsize=12, fontweight='bold')
axes[1].set_ylim([-1.5, 1.5])
axes[1].grid(True, alpha=0.3)

# Electrode B
axes[2].plot(t, electrode_B, color=PRIMARY_GREEN, linewidth=1.5)
axes[2].set_ylabel('Amplitude', fontsize=11)
axes[2].set_xlabel('Time (s)', fontsize=11)
axes[2].set_title('Electrode B (0.5 × source + noise)', fontsize=12, fontweight='bold')
axes[2].set_ylim([-1.5, 1.5])
axes[2].grid(True, alpha=0.3)

# Highlight that they're in phase
for ax in axes[1:]:
    ax.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5)
    ax.axvline(x=1.0, color='gray', linestyle='--', alpha=0.5)

fig.suptitle('Volume Conduction: Same Source at Different Electrodes', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("👁️ Notice: Both electrode signals oscillate IN PHASE!")
print("   They rise and fall together because they see the SAME source.")

In [ ]:
# ============================================================================
# VISUALIZATION 3: Spurious Connectivity Metrics
# ============================================================================

# Compute correlation
correlation = np.corrcoef(electrode_A, electrode_B)[0, 1]

# Compute PLV (Phase Locking Value)
# Extract phases using Hilbert transform
phase_A = np.angle(hilbert(electrode_A))
phase_B = np.angle(hilbert(electrode_B))
phase_diff = phase_A - phase_B
plv = np.abs(np.mean(np.exp(1j * phase_diff)))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

metrics = ['Correlation', 'PLV']
values = [correlation, plv]
colors_bars = [PRIMARY_BLUE, SECONDARY_PURPLE]

for ax, metric, value, color in zip(axes, metrics, values, colors_bars):
    bars = ax.bar([metric], [value], color=color, edgecolor='black', linewidth=2)
    ax.set_ylim([0, 1.1])
    ax.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5)
    ax.set_ylabel('Value', fontsize=12)
    ax.set_title(f'{metric} = {value:.3f}', fontsize=14, fontweight='bold')
    
    # Warning annotation
    ax.annotate('SPURIOUS!', (0, value + 0.05), ha='center', fontsize=12, 
                fontweight='bold', color='red')

fig.suptitle('⚠️ THIS IS NOT REAL CONNECTIVITY! ⚠️', 
             fontsize=16, fontweight='bold', color='red', y=1.02)
plt.tight_layout()
plt.show()

print(f"📊 Results:")
print(f"   Correlation: {correlation:.3f} (very high!)")
print(f"   PLV: {plv:.3f} (near perfect synchronization!)")
print("\n🚨 These high values are ARTIFACTS of volume conduction, not real connectivity!")

---

## 4. The Zero-Lag Signature 🎯

Here's a crucial insight: **volume conduction is nearly instantaneous**.

Electrical fields propagate at close to the speed of light. For the distances involved in EEG (centimeters), the propagation time is essentially zero — far below what we can measure with typical sampling rates.

### The Diagnostic

This gives us a powerful diagnostic:

- **Volume conduction** → signals are correlated with **ZERO time delay**
- **True neural connectivity** → signals have a **non-zero time delay**

Why? Real neural communication involves:
- Axonal conduction: ~1-10 m/s (much slower than light!)
- Synaptic delays: ~0.5-2 ms per synapse
- Processing time in neural circuits

These add up to measurable delays (milliseconds to tens of milliseconds).

### Phase Perspective

In terms of phase:
- **Volume conduction**: phase difference ≈ 0 (or π for inverted polarity)
- **True connectivity**: phase difference ≈ some non-zero value

> 💡 **Key insight**: If the phase difference between two signals is ALWAYS near 0 or π, be very suspicious — it's probably volume conduction!

In [ ]:
# ============================================================================
# FUNCTION 1: Simulate Volume Conduction
# ============================================================================

def simulate_volume_conduction(
    source_signal: NDArray[np.float64],
    weights: list[float],
    noise_level: float = 0.1,
    seed: Optional[int] = None
) -> NDArray[np.float64]:
    """
    Simulate volume conduction: each electrode receives weighted source + noise.
    
    Parameters
    ----------
    source_signal : ndarray of float64
        The underlying brain source signal (1D array).
    weights : list of float
        Mixing weights for each electrode (one per electrode).
    noise_level : float, optional
        Standard deviation of Gaussian noise. Default is 0.1.
    seed : int, optional
        Random seed for reproducibility.
        
    Returns
    -------
    electrode_signals : ndarray of float64
        Simulated electrode signals, shape (n_electrodes, n_samples).
        
    Notes
    -----
    This simulates the simplest case of volume conduction: a single source
    appears at multiple electrodes with different amplitudes but ZERO phase lag.
    """
    if seed is not None:
        np.random.seed(seed)
    
    n_electrodes = len(weights)
    n_samples = len(source_signal)
    
    electrode_signals = np.zeros((n_electrodes, n_samples))
    
    for i, weight in enumerate(weights):
        electrode_signals[i] = weight * source_signal + noise_level * np.random.randn(n_samples)
    
    return electrode_signals


# ============================================================================
# FUNCTION 2: Compute Cross-Correlation
# ============================================================================

def compute_cross_correlation(
    signal1: NDArray[np.float64],
    signal2: NDArray[np.float64],
    max_lag_samples: Optional[int] = None
) -> Tuple[NDArray[np.int64], NDArray[np.float64]]:
    """
    Compute cross-correlation function to identify time lag of maximum correlation.
    
    Parameters
    ----------
    signal1 : ndarray of float64
        First signal (1D array).
    signal2 : ndarray of float64
        Second signal (1D array).
    max_lag_samples : int, optional
        Maximum lag to consider (in samples). If None, uses len(signal)//4.
        
    Returns
    -------
    lags : ndarray of int64
        Lag values in samples.
    correlation : ndarray of float64
        Normalized cross-correlation values.
        
    Notes
    -----
    Positive lag means signal2 leads signal1.
    The peak location indicates the time delay between signals.
    """
    n = len(signal1)
    
    if max_lag_samples is None:
        max_lag_samples = n // 4
    
    # Normalize signals
    s1 = (signal1 - np.mean(signal1)) / np.std(signal1)
    s2 = (signal2 - np.mean(signal2)) / np.std(signal2)
    
    # Full cross-correlation
    xcorr = correlate(s1, s2, mode='full') / n
    
    # Extract relevant portion
    mid = len(xcorr) // 2
    start = mid - max_lag_samples
    end = mid + max_lag_samples + 1
    
    lags = np.arange(-max_lag_samples, max_lag_samples + 1)
    correlation = xcorr[start:end]
    
    return lags, correlation


# Test the functions
print("✓ Functions defined: simulate_volume_conduction, compute_cross_correlation")

In [ ]:
# ============================================================================
# VISUALIZATION 4: Cross-Correlation Shows Zero-Lag
# ============================================================================

# Compute cross-correlation between electrode A and B
lags, xcorr = compute_cross_correlation(electrode_A, electrode_B)
lag_ms = lags / fs * 1000  # Convert to milliseconds

# Find peak
peak_idx = np.argmax(xcorr)
peak_lag_ms = lag_ms[peak_idx]
peak_value = xcorr[peak_idx]

# Plot
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(lag_ms, xcorr, color=PRIMARY_BLUE, linewidth=2)
ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5, label='Zero lag')
ax.axvline(x=peak_lag_ms, color=PRIMARY_RED, linestyle='-', linewidth=2, 
           label=f'Peak at {peak_lag_ms:.1f} ms')

# Mark the peak
ax.plot(peak_lag_ms, peak_value, 'o', markersize=12, color=PRIMARY_RED, 
        markeredgecolor='darkred', markeredgewidth=2, zorder=10)

ax.set_xlabel('Lag (ms)', fontsize=12)
ax.set_ylabel('Cross-Correlation', fontsize=12)
ax.set_title('Cross-Correlation: Peak at Zero Lag = Volume Conduction Signature', 
             fontsize=13, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

# Annotation
ax.annotate(f'Peak at {peak_lag_ms:.1f} ms\n(effectively zero!)', 
            xy=(peak_lag_ms, peak_value), xytext=(50, peak_value - 0.1),
            fontsize=11, ha='left',
            arrowprops=dict(arrowstyle='->', color='red'),
            bbox=dict(boxstyle='round', facecolor='lightyellow'))

plt.tight_layout()
plt.show()

print(f"📊 Cross-correlation peak at lag = {peak_lag_ms:.2f} ms")
print("   This is effectively ZERO — the hallmark of volume conduction!")

In [ ]:
# ============================================================================
# VISUALIZATION 5: Phase Difference Distribution
# ============================================================================

# Phase difference from earlier (already computed)
# Wrap to [-π, π]
phase_diff_wrapped = np.angle(np.exp(1j * phase_diff))

# Create histogram
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Histogram
axes[0].hist(phase_diff_wrapped, bins=50, color=SECONDARY_PURPLE, 
             edgecolor='black', alpha=0.7, density=True)
axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero phase diff')
axes[0].axvline(x=np.pi, color='orange', linestyle='--', linewidth=2, alpha=0.7)
axes[0].axvline(x=-np.pi, color='orange', linestyle='--', linewidth=2, alpha=0.7, 
                label='±π (polarity inversion)')
axes[0].set_xlabel('Phase Difference (radians)', fontsize=12)
axes[0].set_ylabel('Density', fontsize=12)
axes[0].set_title('Phase Difference Distribution', fontsize=12, fontweight='bold')
axes[0].set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
axes[0].set_xticklabels(['-π', '-π/2', '0', 'π/2', 'π'])
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Polar histogram
ax_polar = fig.add_subplot(122, projection='polar')
bins_polar = np.linspace(-np.pi, np.pi, 37)
hist, bin_edges = np.histogram(phase_diff_wrapped, bins=bins_polar, density=True)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
width = bin_edges[1] - bin_edges[0]

bars = ax_polar.bar(bin_centers, hist, width=width, color=SECONDARY_PURPLE, 
                    edgecolor='black', alpha=0.7)
ax_polar.set_title('Polar View\n(concentrated near 0)', fontsize=11, fontweight='bold')

fig.suptitle('Phase Difference Locked Near Zero → Volume Conduction!', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Statistics
mean_phase = np.angle(np.mean(np.exp(1j * phase_diff_wrapped)))
std_phase = np.std(phase_diff_wrapped)
print(f"📐 Phase difference statistics:")
print(f"   Mean: {np.degrees(mean_phase):.1f}° (near zero!)")
print(f"   Std: {np.degrees(std_phase):.1f}° (very narrow distribution)")

---

## 5. Why This Matters for Hyperscanning 🧠🧠

Now let's connect this to the main topic of our workshop: **hyperscanning** (simultaneous recording from multiple brains).

### The Good News

In hyperscanning, we compare brain signals from **different people**. Volume conduction is an electrical phenomenon — it requires a physical conductive path.

**There is NO conductive path between two separate heads!**

This means:
- Electrical fields from Person A's brain cannot reach Person B's electrodes
- Inter-brain connectivity is **immune to volume conduction**
- If we see correlation between two people's brain signals, it's NOT an artifact

### The Bad News

Volume conduction still affects **within-participant** analysis:
- Comparing electrodes on the SAME head still has the problem
- Intra-brain connectivity is just as problematic as in single-brain studies

Also, if we're sloppy with our analysis:
- Comparing the "wrong" electrode pairs unknowingly
- Not properly accounting for reference electrode effects

### The Key Advantage

> 🎯 **Hyperscanning Advantage**: Inter-brain connectivity is "cleaner" than intra-brain connectivity because volume conduction cannot occur between separate heads.

This is one reason why hyperscanning is so valuable — we can be more confident that inter-brain synchronization reflects true neural coupling!

In [ ]:
# ============================================================================
# VISUALIZATION 6: Two-Brain Hyperscanning Setup
# ============================================================================

fig, ax = plt.subplots(figsize=(14, 8))

# Draw two heads
for x_center, color, label in [(0.25, SUBJECT_1, 'Participant 1'), 
                                 (0.75, SUBJECT_2, 'Participant 2')]:
    # Head
    head = Ellipse((x_center, 0.5), 0.35, 0.45, fill=True, 
                   facecolor=color, alpha=0.2, edgecolor=color, linewidth=3)
    ax.add_patch(head)
    
    # Brain region
    brain = Ellipse((x_center, 0.47), 0.25, 0.3, fill=True, 
                    facecolor=color, alpha=0.3, edgecolor=color, linewidth=1)
    ax.add_patch(brain)
    
    # Source in brain
    ax.plot(x_center, 0.47, 'o', markersize=10, color=color, 
            markeredgecolor='black', markeredgewidth=1.5)
    
    # Electrodes
    for dx, dy in [(-0.1, 0.15), (0, 0.2), (0.1, 0.15)]:
        ax.plot(x_center + dx, 0.5 + dy, 's', markersize=8, color=color,
                markeredgecolor='black', markeredgewidth=1)
    
    ax.annotate(label, (x_center, 0.15), ha='center', fontsize=12, 
                fontweight='bold', color=color)

# Within-brain connections (problematic)
ax.annotate('', xy=(0.15, 0.65), xytext=(0.35, 0.65),
            arrowprops=dict(arrowstyle='<->', color='red', lw=2))
ax.annotate('WITHIN-BRAIN\n⚠️ Volume conduction!', (0.25, 0.78), 
            ha='center', fontsize=10, color='red', fontweight='bold')

ax.annotate('', xy=(0.65, 0.65), xytext=(0.85, 0.65),
            arrowprops=dict(arrowstyle='<->', color='red', lw=2))

# Between-brain connection (clean)
ax.annotate('', xy=(0.42, 0.5), xytext=(0.58, 0.5),
            arrowprops=dict(arrowstyle='<->', color='green', lw=3))
ax.annotate('BETWEEN-BRAIN\n✓ No volume conduction!', (0.5, 0.35), 
            ha='center', fontsize=11, color='green', fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

# Big X for no volume conduction between heads
ax.plot([0.42, 0.58], [0.55, 0.45], 'g-', linewidth=2, alpha=0.5)
ax.plot([0.42, 0.58], [0.45, 0.55], 'g-', linewidth=2, alpha=0.5)

ax.set_xlim(0, 1)
ax.set_ylim(0.1, 0.9)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Hyperscanning: Inter-Brain Connectivity is Cleaner!', 
             fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("🧠🧠 Key insight for hyperscanning:")
print("   - Within-brain connectivity: Still affected by volume conduction")
print("   - Between-brain connectivity: NO volume conduction possible!")
print("   - This makes inter-brain synchronization more trustworthy.")

---

## 6. Demonstrating the Problem — Multi-Source Simulation

Let's create a more realistic simulation with multiple sources and electrodes. This will show how volume conduction creates patterns of spurious connectivity across the scalp.

In [ ]:
# ============================================================================
# FUNCTION 3: Create Mixing Matrix
# ============================================================================

def create_mixing_matrix(
    n_sources: int,
    n_electrodes: int,
    spread: float = 0.5,
    seed: Optional[int] = None
) -> NDArray[np.float64]:
    """
    Create a mixing matrix simulating field spread from sources to electrodes.
    
    Parameters
    ----------
    n_sources : int
        Number of brain sources.
    n_electrodes : int
        Number of scalp electrodes.
    spread : float, optional
        Controls how much sources spread (0 = no spread, 1 = full spread).
        Default is 0.5.
    seed : int, optional
        Random seed for reproducibility.
        
    Returns
    -------
    mixing_matrix : ndarray of float64
        Mixing matrix, shape (n_electrodes, n_sources).
        Each row shows how much each source contributes to that electrode.
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Create base pattern: each electrode mainly sees nearby sources
    # but also has some spread to distant sources
    mixing_matrix = np.zeros((n_electrodes, n_sources))
    
    for i in range(n_electrodes):
        for j in range(n_sources):
            # Distance-based weight (closer = stronger)
            # Normalized positions
            electrode_pos = i / (n_electrodes - 1) if n_electrodes > 1 else 0.5
            source_pos = j / (n_sources - 1) if n_sources > 1 else 0.5
            distance = abs(electrode_pos - source_pos)
            
            # Weight decreases with distance, controlled by spread parameter
            weight = np.exp(-distance / (spread + 0.1))
            mixing_matrix[i, j] = weight
    
    # Add some random variation
    mixing_matrix += 0.1 * np.random.rand(n_electrodes, n_sources)
    
    # Normalize rows to sum to 1
    mixing_matrix = mixing_matrix / mixing_matrix.sum(axis=1, keepdims=True)
    
    return mixing_matrix


def apply_mixing(
    sources: NDArray[np.float64],
    mixing_matrix: NDArray[np.float64],
    noise_level: float = 0.1
) -> NDArray[np.float64]:
    """
    Apply mixing matrix to simulate scalp EEG from brain sources.
    
    Parameters
    ----------
    sources : ndarray of float64
        Source signals, shape (n_sources, n_samples).
    mixing_matrix : ndarray of float64
        Mixing matrix, shape (n_electrodes, n_sources).
    noise_level : float, optional
        Standard deviation of sensor noise. Default is 0.1.
        
    Returns
    -------
    electrodes : ndarray of float64
        Electrode signals, shape (n_electrodes, n_samples).
    """
    electrodes = mixing_matrix @ sources
    electrodes += noise_level * np.random.randn(*electrodes.shape)
    return electrodes


print("✓ Functions defined: create_mixing_matrix, apply_mixing")

In [ ]:
# ============================================================================
# VISUALIZATION 7: Multi-Source Simulation
# ============================================================================

# Create 3 independent brain sources
n_sources = 3
n_electrodes = 6
duration = 3.0
t = np.arange(0, duration, 1/fs)
n_samples = len(t)

# Independent oscillations at different frequencies
np.random.seed(42)
sources = np.zeros((n_sources, n_samples))
source_freqs = [8, 12, 18]  # Alpha, high-alpha, beta
source_colors = [PRIMARY_BLUE, PRIMARY_GREEN, SECONDARY_ORANGE]

for i, freq in enumerate(source_freqs):
    sources[i] = np.sin(2 * np.pi * freq * t + np.random.rand() * 2 * np.pi)

# Create mixing matrix with moderate spread
mixing_matrix = create_mixing_matrix(n_sources, n_electrodes, spread=0.4, seed=42)

# Apply mixing
electrodes = apply_mixing(sources, mixing_matrix, noise_level=0.1)

# Plot
fig = plt.figure(figsize=(14, 10))
gs = fig.add_gridspec(3, 2, width_ratios=[1, 1.2], height_ratios=[0.8, 1, 1])

# Mixing matrix heatmap
ax_mix = fig.add_subplot(gs[0, 0])
im = ax_mix.imshow(mixing_matrix, cmap='YlOrRd', aspect='auto')
ax_mix.set_xlabel('Source', fontsize=11)
ax_mix.set_ylabel('Electrode', fontsize=11)
ax_mix.set_xticks(range(n_sources))
ax_mix.set_xticklabels([f'S{i+1}\n({f}Hz)' for i, f in enumerate(source_freqs)])
ax_mix.set_yticks(range(n_electrodes))
ax_mix.set_yticklabels([f'E{i+1}' for i in range(n_electrodes)])
ax_mix.set_title('Mixing Matrix\n(how much each source → electrode)', fontsize=11, fontweight='bold')
plt.colorbar(im, ax=ax_mix, label='Weight')

# Source signals
ax_src = fig.add_subplot(gs[0, 1])
for i in range(n_sources):
    offset = (n_sources - 1 - i) * 2.5
    ax_src.plot(t[:256], sources[i, :256] + offset, color=source_colors[i], 
                linewidth=1.5, label=f'Source {i+1} ({source_freqs[i]} Hz)')
ax_src.set_ylabel('Sources (offset)', fontsize=11)
ax_src.set_title('Independent Brain Sources', fontsize=11, fontweight='bold')
ax_src.legend(loc='upper right', fontsize=9)
ax_src.set_xlim([0, 1])
ax_src.grid(True, alpha=0.3)

# Electrode signals
ax_elec = fig.add_subplot(gs[1, :])
electrode_colors = plt.cm.viridis(np.linspace(0.2, 0.8, n_electrodes))
for i in range(n_electrodes):
    offset = (n_electrodes - 1 - i) * 2
    ax_elec.plot(t[:512], electrodes[i, :512] + offset, color=electrode_colors[i], 
                 linewidth=1, label=f'E{i+1}')
ax_elec.set_xlabel('Time (s)', fontsize=11)
ax_elec.set_ylabel('Electrodes (offset)', fontsize=11)
ax_elec.set_title('Resulting Electrode Signals (Mixtures of Sources)', fontsize=11, fontweight='bold')
ax_elec.legend(loc='upper right', fontsize=9, ncol=2)
ax_elec.set_xlim([0, 2])
ax_elec.grid(True, alpha=0.3)

# Annotation
ax_note = fig.add_subplot(gs[2, :])
ax_note.text(0.5, 0.5, 
             "Each electrode signal is a MIXTURE of all sources!\n"
             "Nearby electrodes have similar mixtures → spurious connectivity",
             ha='center', va='center', fontsize=12,
             bbox=dict(boxstyle='round', facecolor='lightyellow', edgecolor='orange', linewidth=2))
ax_note.axis('off')

fig.suptitle('Volume Conduction Simulation: Sources → Mixing → Electrodes', 
             fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# VISUALIZATION 8: Spurious Connectivity Matrix
# ============================================================================

# Compute connectivity matrix (correlation) between all electrode pairs
connectivity_matrix = np.corrcoef(electrodes)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Correlation matrix
im = axes[0].imshow(connectivity_matrix, cmap='RdBu_r', vmin=-1, vmax=1, aspect='equal')
axes[0].set_xticks(range(n_electrodes))
axes[0].set_yticks(range(n_electrodes))
axes[0].set_xticklabels([f'E{i+1}' for i in range(n_electrodes)])
axes[0].set_yticklabels([f'E{i+1}' for i in range(n_electrodes)])
axes[0].set_title('Correlation Matrix\n(Spurious Connectivity!)', fontsize=12, fontweight='bold')
plt.colorbar(im, ax=axes[0], label='Correlation')

# Add values
for i in range(n_electrodes):
    for j in range(n_electrodes):
        if i != j:
            axes[0].text(j, i, f'{connectivity_matrix[i,j]:.2f}', 
                        ha='center', va='center', fontsize=9,
                        color='white' if abs(connectivity_matrix[i,j]) > 0.5 else 'black')

# Mixing matrix similarity (explains the pattern)
mixing_similarity = mixing_matrix @ mixing_matrix.T
im2 = axes[1].imshow(mixing_similarity, cmap='YlOrRd', aspect='equal')
axes[1].set_xticks(range(n_electrodes))
axes[1].set_yticks(range(n_electrodes))
axes[1].set_xticklabels([f'E{i+1}' for i in range(n_electrodes)])
axes[1].set_yticklabels([f'E{i+1}' for i in range(n_electrodes)])
axes[1].set_title('Mixing Similarity\n(Shared Sources)', fontsize=12, fontweight='bold')
plt.colorbar(im2, ax=axes[1], label='Similarity')

fig.suptitle('High Correlation = Shared Sources (NOT Real Connectivity!)', 
             fontsize=14, fontweight='bold', color='red', y=1.02)
plt.tight_layout()
plt.show()

print("📊 Notice: Nearby electrodes (E1-E2, E5-E6) have HIGH correlation.")
print("   This is because they share the SAME sources — volume conduction!")
print("   The pattern matches the mixing similarity matrix.")

---

## 7. Solutions Overview: How Do We Fix This?

The volume conduction problem seems insurmountable, but researchers have developed several clever strategies to mitigate its effects. These solutions fall into **three main categories**:

### Category 1: Metrics Insensitive to Zero-Lag
Since volume conduction creates **instantaneous** (zero-lag) mixing, we can design connectivity metrics that **ignore zero-lag synchronization**.

| Metric | Strategy | Key Idea |
|--------|----------|----------|
| **Imaginary Coherence (ImCoh)** | Uses only the imaginary part of coherence | Zero-lag = real-valued, so imaginary part = 0 |
| **Phase Lag Index (PLI)** | Measures consistency of phase lead/lag | Zero-lag = 0° difference, not counted |
| **weighted PLI (wPLI)** | Weighted version of PLI | More robust to noise |

### Category 2: Spatial Filtering
Instead of changing the metric, we can **transform the data** to reduce mixing before computing connectivity.

| Method | Strategy |
|--------|----------|
| **Laplacian Reference** | Emphasizes local activity, reduces spread |
| **Source Localization** | Reconstructs original sources from sensor data |
| **ICA** | Separates mixed signals into independent components |

### Category 3: Statistical Approaches
Use statistical methods to distinguish true from spurious connectivity.

| Method | Strategy |
|--------|----------|
| **Surrogate Testing** | Compare to null distribution without true coupling |
| **Cross-frequency analysis** | True interactions often cross frequency bands |

> 🎯 **In this workshop, we'll focus on Category 1 metrics** (ImCoh, PLI, wPLI) as they are most commonly used in hyperscanning research.

In [ ]:
# ============================================================================
# VISUALIZATION 9: Decision Flowchart for Metric Selection
# ============================================================================

fig, ax = plt.subplots(figsize=(15, 10))
ax.set_xlim(0, 16)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('Decision Flowchart: Choosing Volume Conduction-Robust Metrics', 
             fontsize=14, fontweight='bold', pad=20)

# Helper function for boxes
def draw_box(ax, x, y, w, h, text, color, text_color='white', fontsize=10):
    box = plt.Rectangle((x - w/2, y - h/2), w, h, 
                        facecolor=color, edgecolor='black', linewidth=2,
                        alpha=0.9, zorder=2)
    ax.add_patch(box)
    ax.text(x, y, text, ha='center', va='center', fontsize=fontsize,
            fontweight='bold', color=text_color, zorder=3,
            wrap=True)
    return (x, y)

def draw_arrow(ax, start, end, color='black'):
    ax.annotate('', xy=end, xytext=start,
                arrowprops=dict(arrowstyle='->', color=color, lw=2),
                zorder=1)

# Start box
draw_box(ax, 7, 9, 4, 1, 'EEG Connectivity\nAnalysis', PRIMARY_BLUE)

# Question 1
draw_box(ax, 7, 7, 5, 1, 'Concerned about\nvolume conduction?', SECONDARY_ORANGE, 'black')
draw_arrow(ax, (7, 8.5), (7, 7.5))

# No path
draw_box(ax, 3, 7, 2.5, 0.8, 'No', '#888888')
draw_arrow(ax, (4.5, 7), (4.25, 7))
draw_box(ax, 3, 5.5, 3, 1, 'Standard metrics:\nCoherence, PLV', '#666666')
draw_arrow(ax, (3, 6.4), (3, 6))

# Yes path  
draw_box(ax, 11, 7, 2.5, 0.8, 'Yes', PRIMARY_GREEN)
draw_arrow(ax, (9.5, 7), (9.75, 7))

# Category selection
draw_box(ax, 11, 5.5, 4.5, 1, 'Choose approach:', SECONDARY_PURPLE)
draw_arrow(ax, (11, 6.4), (11, 6))

# Three solutions
draw_box(ax, 7, 3.5, 3.5, 1.2, 'Robust Metrics\n(ImCoh, PLI, wPLI)', PRIMARY_BLUE)
draw_box(ax, 11, 3.5, 3.5, 1.2, 'Spatial Filtering\n(Laplacian, ICA)', PRIMARY_RED)
draw_box(ax, 14, 3.5, 2.5, 1.2, 'Statistical\nControls', PRIMARY_GREEN, 'black')

draw_arrow(ax, (9.25, 5), (7, 4.1))
draw_arrow(ax, (11, 5), (11, 4.1))
draw_arrow(ax, (12.75, 5), (14, 4.1))

# Pros/cons
ax.text(7, 2.2, '✓ Easy to implement\n✓ Well understood\n✗ May miss true 0-lag', 
        ha='center', va='top', fontsize=9, style='italic')
ax.text(11, 2.2, '✓ Reduces mixing\n✓ Better localization\n✗ Requires expertise', 
        ha='center', va='top', fontsize=9, style='italic')
ax.text(14, 2.2, '✓ Rigorous\n✗ Complex\n✗ Computationally heavy', 
        ha='center', va='top', fontsize=9, style='italic')

# Recommendation box
rec_box = plt.Rectangle((5, 0.3), 8, 1.2, facecolor='#ffffcc', 
                         edgecolor=SECONDARY_ORANGE, linewidth=3, zorder=2)
ax.add_patch(rec_box)
ax.text(9, 0.9, '💡 Recommended for Hyperscanning: Start with PLI or wPLI', 
        ha='center', va='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

---

## 8. Imaginary Coherence: Ignoring the Real Part

**Imaginary Coherence (ImCoh)** exploits a fundamental property of volume conduction: **instantaneous mixing produces only real-valued coherence**.

### The Mathematical Insight

Standard **coherence** is a complex number:
$$\text{Coh}_{xy}(f) = \frac{S_{xy}(f)}{\sqrt{S_{xx}(f) \cdot S_{yy}(f)}}$$

This can be decomposed into:
- **Real part**: Captures in-phase (0°) and anti-phase (180°) relationships
- **Imaginary part**: Captures phase-shifted relationships (≠ 0° or 180°)

### Why Volume Conduction is Real-Valued

When signal $y$ is a linear mixture of signal $x$:
$$y(t) = \alpha \cdot x(t) + \text{noise}$$

The cross-spectrum $S_{xy}$ is **purely real** because:
- Same signal at same time → phase difference = 0°
- cos(0°) = 1, sin(0°) = 0
- Real part ≠ 0, but **Imaginary part = 0**

### The Solution

**Imaginary Coherence** uses only the imaginary part:
$$\text{ImCoh}_{xy}(f) = \text{Im}\left(\text{Coh}_{xy}(f)\right)$$

This is **insensitive to zero-lag mixing** because volume conduction contributes nothing to the imaginary part!

In [ ]:
# ============================================================================
# VISUALIZATION 10: Complex Plane - Understanding Imaginary Coherence
# ============================================================================

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Helper: draw complex plane
def setup_complex_plane(ax, title):
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.axvline(x=0, color='black', linewidth=0.5)
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.set_xlabel('Real Part', fontsize=11)
    ax.set_ylabel('Imaginary Part', fontsize=11)
    ax.set_title(title, fontsize=12, fontweight='bold')
    # Unit circle
    theta = np.linspace(0, 2*np.pi, 100)
    ax.plot(np.cos(theta), np.sin(theta), 'k--', alpha=0.3, linewidth=1)

# Panel 1: Volume conduction (real-valued coherence)
ax = axes[0]
setup_complex_plane(ax, 'Volume Conduction\n(Zero-Lag)')

# Show multiple "coherence" values on real axis
for val in [0.3, 0.6, 0.85]:
    ax.arrow(0, 0, val, 0, head_width=0.08, head_length=0.05, 
             fc=PRIMARY_RED, ec=PRIMARY_RED, linewidth=2)
ax.scatter([0.3, 0.6, 0.85], [0, 0, 0], s=100, c=PRIMARY_RED, zorder=5)

# Highlight real axis
ax.fill_between([-1.5, 1.5], [-0.1, -0.1], [0.1, 0.1], 
                color=PRIMARY_RED, alpha=0.2)
ax.text(0.7, -0.5, 'All values\non real axis!', fontsize=10, 
        ha='center', color=PRIMARY_RED, fontweight='bold')
ax.text(0, 1.2, 'ImCoh = 0', fontsize=14, ha='center', 
        color=PRIMARY_RED, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='white', edgecolor=PRIMARY_RED))

# Panel 2: True connectivity (complex coherence)
ax = axes[1]
setup_complex_plane(ax, 'True Connectivity\n(Phase Lag ≠ 0)')

# Show coherence vectors with different phases
phases = [np.pi/6, np.pi/3, np.pi/2, 2*np.pi/3]
magnitudes = [0.7, 0.8, 0.6, 0.75]
for phase, mag in zip(phases, magnitudes):
    x, y = mag * np.cos(phase), mag * np.sin(phase)
    ax.arrow(0, 0, x*0.9, y*0.9, head_width=0.08, head_length=0.05,
             fc=PRIMARY_BLUE, ec=PRIMARY_BLUE, linewidth=2)
    ax.scatter([x], [y], s=100, c=PRIMARY_BLUE, zorder=5)

# Highlight imaginary axis
ax.fill_betweenx([-1.5, 1.5], [-0.1, -0.1], [0.1, 0.1],
                 color=PRIMARY_BLUE, alpha=0.2)
ax.text(0.8, 0.8, 'Non-zero\nimaginary\ncomponents!', fontsize=10,
        ha='center', color=PRIMARY_BLUE, fontweight='bold')
ax.text(0, 1.2, 'ImCoh ≠ 0', fontsize=14, ha='center',
        color=PRIMARY_BLUE, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='white', edgecolor=PRIMARY_BLUE))

# Panel 3: What ImCoh captures
ax = axes[2]
setup_complex_plane(ax, 'Imaginary Coherence\nCaptures Only This')

# Gray out real axis
ax.fill_between([-1.5, 1.5], [-0.15, -0.15], [0.15, 0.15],
                color='gray', alpha=0.3)
ax.text(0.8, 0, '✗ Ignored', fontsize=10, color='gray', va='center')

# Highlight imaginary axis (positive and negative)
ax.fill_betweenx([0.15, 1.5], [-0.15, -0.15], [0.15, 0.15],
                 color=PRIMARY_GREEN, alpha=0.3)
ax.fill_betweenx([-1.5, -0.15], [-0.15, -0.15], [0.15, 0.15],
                 color=PRIMARY_GREEN, alpha=0.3)

# Show projection
phase = np.pi/3
mag = 0.8
x, y = mag * np.cos(phase), mag * np.sin(phase)
ax.arrow(0, 0, x*0.9, y*0.9, head_width=0.08, head_length=0.05,
         fc=PRIMARY_BLUE, ec=PRIMARY_BLUE, linewidth=2, alpha=0.5)
ax.plot([x, x], [0, y], 'g--', linewidth=2)
ax.plot([0, x], [y, y], 'g--', linewidth=2)
ax.scatter([0], [y], s=150, c=PRIMARY_GREEN, zorder=5, marker='*')
ax.text(0.3, y, f'ImCoh = {y:.2f}', fontsize=11, color=PRIMARY_GREEN, fontweight='bold')

plt.tight_layout()
plt.show()

print("Key Insight: Imaginary Coherence is blind to volume conduction!")
print("It only captures phase relationships that are NOT 0° or 180°.")

---

## 9. Phase Lag Index (PLI): Counting Lead vs Lag

**Phase Lag Index (PLI)** takes a different approach: instead of looking at coherence magnitude, it focuses on **phase differences**.

### The Core Idea

At each time point, we ask: **"Is signal A ahead or behind signal B?"**

- If A leads B consistently → **positive phase difference**
- If B leads A consistently → **negative phase difference**  
- If volume conduction → **zero phase difference** (neither leads)

### The Mathematical Definition

$$\text{PLI} = \left| \langle \text{sign}(\Delta\phi(t)) \rangle \right|$$

Where:
- $\Delta\phi(t)$ is the instantaneous phase difference
- $\text{sign}()$ returns +1, 0, or -1
- $\langle \cdot \rangle$ is the average over time

### Why It Works

| Scenario | Phase Difference | sign(Δφ) | PLI |
|----------|------------------|----------|-----|
| Volume conduction | Always ~0° | ~0 | **0** |
| A consistently leads B | Mostly positive | Mostly +1 | **High** |
| B consistently leads A | Mostly negative | Mostly -1 | **High** |
| Random relationship | Mixed positive/negative | Cancels out | **Low** |

> 💡 **Key advantage**: PLI is also robust to **amplitude differences** between signals!

In [ ]:
# ============================================================================
# VISUALIZATION 11: PLI Intuition - Sign of Phase Difference
# ============================================================================

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Generate example signals with different phase relationships
np.random.seed(42)
t = np.arange(0, 2, 1/fs)
freq = 10  # Hz

# Scenario 1: Volume conduction (same signal)
sig1_vc = np.sin(2 * np.pi * freq * t)
sig2_vc = 0.8 * sig1_vc + 0.1 * np.random.randn(len(t))

# Scenario 2: True lead (signal 2 leads by pi/4)
sig1_lead = np.sin(2 * np.pi * freq * t)
sig2_lead = np.sin(2 * np.pi * freq * t + np.pi/4) + 0.1 * np.random.randn(len(t))

# Scenario 3: Random phase relationship
sig1_rand = np.sin(2 * np.pi * freq * t)
phase_drift = np.cumsum(0.1 * np.random.randn(len(t)))
sig2_rand = np.sin(2 * np.pi * freq * t + phase_drift)

scenarios = [
    (sig1_vc, sig2_vc, 'Volume Conduction', PRIMARY_RED),
    (sig1_lead, sig2_lead, 'True Connectivity (B leads)', PRIMARY_BLUE),
    (sig1_rand, sig2_rand, 'Random Relationship', SECONDARY_ORANGE)
]

for i, (s1, s2, title, color) in enumerate(scenarios):
    # Top row: Time series
    ax = axes[0, i]
    ax.plot(t[:256], s1[:256], label='Signal A', color=PRIMARY_BLUE, linewidth=1.5)
    ax.plot(t[:256], s2[:256], label='Signal B', color=PRIMARY_RED, linewidth=1.5)
    ax.set_title(title, fontsize=12, fontweight='bold', color=color)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.legend(loc='upper right', fontsize=9)
    ax.set_xlim(0, 1)
    
    # Bottom row: Sign of phase difference over time
    ax = axes[1, i]
    
    # Compute phase difference
    analytic1 = scipy.signal.hilbert(s1)
    analytic2 = scipy.signal.hilbert(s2)
    phase1 = np.angle(analytic1)
    phase2 = np.angle(analytic2)
    phase_diff = phase2 - phase1
    phase_diff = np.arctan2(np.sin(phase_diff), np.cos(phase_diff))  # Wrap to [-pi, pi]
    
    signs = np.sign(phase_diff)
    
    # Plot as colored bars
    for j in range(len(t)-1):
        if signs[j] > 0:
            ax.axvspan(t[j], t[j+1], color=PRIMARY_GREEN, alpha=0.7)
        elif signs[j] < 0:
            ax.axvspan(t[j], t[j+1], color=PRIMARY_RED, alpha=0.7)
        else:
            ax.axvspan(t[j], t[j+1], color='gray', alpha=0.5)
    
    # Compute PLI
    pli = np.abs(np.mean(signs))
    
    ax.axhline(y=0, color='black', linewidth=2)
    ax.set_xlim(0, 2)
    ax.set_ylim(-0.5, 0.5)
    ax.set_xlabel('Time (s)')
    ax.set_yticks([])
    ax.set_ylabel('sign(Δφ)')
    
    # Add legend
    ax.text(0.05, 0.35, '+1 (B leads)', fontsize=10, color=PRIMARY_GREEN, fontweight='bold')
    ax.text(0.05, -0.35, '-1 (A leads)', fontsize=10, color=PRIMARY_RED, fontweight='bold')
    
    # PLI value
    ax.text(1.5, 0.35, f'PLI = {pli:.2f}', fontsize=14, fontweight='bold',
            ha='center', bbox=dict(boxstyle='round', facecolor='white', edgecolor=color))

plt.tight_layout()
plt.show()

print("Interpretation:")
print("• Volume Conduction: PLI ≈ 0 (phases are identical, sign is randomly ±1 due to noise)")
print("• True Connectivity: PLI > 0 (consistent lead/lag relationship)")
print("• Random: PLI ≈ 0 (positive and negative cancel out)")

---

## 10. PLV vs PLI: The Critical Comparison

Now let's directly compare **PLV** (vulnerable to volume conduction) with **PLI** (robust to volume conduction) using the same data.

This is perhaps the most important visualization in understanding why metric choice matters!

In [ ]:
# ============================================================================
# FUNCTIONS 5-6: PLI Implementation and Comparison Functions
# ============================================================================

def compute_pli(
    signal_1: NDArray[np.floating],
    signal_2: NDArray[np.floating]
) -> np.floating:
    """
    Compute Phase Lag Index between two signals.
    
    PLI measures the asymmetry of the phase difference distribution.
    It is robust to volume conduction because zero-lag mixing produces
    symmetric phase differences (around 0), leading to PLI = 0.
    
    Parameters
    ----------
    signal_1 : NDArray[np.floating]
        First input signal.
    signal_2 : NDArray[np.floating]
        Second input signal.
        
    Returns
    -------
    np.floating
        Phase Lag Index value between 0 and 1.
        0 = no consistent lead/lag (or volume conduction)
        1 = perfect consistent lead/lag relationship
    """
    # Get instantaneous phases via Hilbert transform
    analytic_1 = scipy.signal.hilbert(signal_1)
    analytic_2 = scipy.signal.hilbert(signal_2)
    
    phase_1 = np.angle(analytic_1)
    phase_2 = np.angle(analytic_2)
    
    # Compute phase difference
    phase_diff = phase_2 - phase_1
    
    # Wrap to [-pi, pi]
    phase_diff = np.arctan2(np.sin(phase_diff), np.cos(phase_diff))
    
    # PLI = |mean(sign(phase_diff))|
    pli = np.abs(np.mean(np.sign(phase_diff)))
    
    return pli


def simulate_volume_conduction_scenario(
    fs: int = 256,
    duration: float = 5.0,
    freq: float = 10.0,
    mixing_strength: float = 0.5,
    noise_level: float = 0.1,
    seed: Optional[int] = None
) -> Tuple[NDArray[np.floating], NDArray[np.floating], NDArray[np.floating]]:
    """
    Simulate a volume conduction scenario with one source and two electrodes.
    
    Parameters
    ----------
    fs : int
        Sampling frequency in Hz.
    duration : float
        Duration in seconds.
    freq : float
        Frequency of the source signal in Hz.
    mixing_strength : float
        How much the source contributes to electrode 2 (0 to 1).
    noise_level : float
        Standard deviation of added noise.
    seed : Optional[int]
        Random seed for reproducibility.
        
    Returns
    -------
    Tuple[NDArray, NDArray, NDArray]
        Time vector, electrode 1 signal, electrode 2 signal.
    """
    if seed is not None:
        np.random.seed(seed)
    
    t = np.arange(0, duration, 1/fs)
    
    # Source signal
    source = np.sin(2 * np.pi * freq * t)
    
    # Electrode signals (both receive the same source, different weights)
    electrode_1 = source + noise_level * np.random.randn(len(t))
    electrode_2 = mixing_strength * source + noise_level * np.random.randn(len(t))
    
    return t, electrode_1, electrode_2


def simulate_true_connectivity_scenario(
    fs: int = 256,
    duration: float = 5.0,
    freq: float = 10.0,
    phase_lag: float = np.pi/4,
    noise_level: float = 0.1,
    seed: Optional[int] = None
) -> Tuple[NDArray[np.floating], NDArray[np.floating], NDArray[np.floating]]:
    """
    Simulate a true connectivity scenario with consistent phase lag.
    
    Parameters
    ----------
    fs : int
        Sampling frequency in Hz.
    duration : float
        Duration in seconds.
    freq : float
        Frequency of the oscillation in Hz.
    phase_lag : float
        Phase lag between signals in radians.
    noise_level : float
        Standard deviation of added noise.
    seed : Optional[int]
        Random seed for reproducibility.
        
    Returns
    -------
    Tuple[NDArray, NDArray, NDArray]
        Time vector, signal 1, signal 2.
    """
    if seed is not None:
        np.random.seed(seed)
    
    t = np.arange(0, duration, 1/fs)
    
    # Two signals with consistent phase lag
    signal_1 = np.sin(2 * np.pi * freq * t) + noise_level * np.random.randn(len(t))
    signal_2 = np.sin(2 * np.pi * freq * t + phase_lag) + noise_level * np.random.randn(len(t))
    
    return t, signal_1, signal_2


print("Functions defined:")
print("• compute_pli(signal_1, signal_2) → Phase Lag Index")
print("• simulate_volume_conduction_scenario(...) → Spurious connectivity scenario")
print("• simulate_true_connectivity_scenario(...) → True connectivity scenario")

In [ ]:
# ============================================================================
# VISUALIZATION 12: PLV vs PLI Comparison - The Key Insight
# ============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Scenario A: Volume Conduction
t_vc, sig1_vc, sig2_vc = simulate_volume_conduction_scenario(
    fs=fs, duration=5.0, freq=10.0, mixing_strength=0.7, noise_level=0.1, seed=42
)

# Scenario B: True Connectivity  
t_tc, sig1_tc, sig2_tc = simulate_true_connectivity_scenario(
    fs=fs, duration=5.0, freq=10.0, phase_lag=np.pi/4, noise_level=0.1, seed=42
)

# Compute PLV for both (using our earlier compute_plv_simple function logic)
def compute_plv_local(s1, s2):
    a1 = scipy.signal.hilbert(s1)
    a2 = scipy.signal.hilbert(s2)
    phase_diff = np.angle(a2) - np.angle(a1)
    return np.abs(np.mean(np.exp(1j * phase_diff)))

# Calculate metrics
plv_vc = compute_plv_local(sig1_vc, sig2_vc)
pli_vc = compute_pli(sig1_vc, sig2_vc)
plv_tc = compute_plv_local(sig1_tc, sig2_tc)
pli_tc = compute_pli(sig1_tc, sig2_tc)

# Top Left: Volume Conduction - Time series
ax = axes[0, 0]
ax.plot(t_vc[:512], sig1_vc[:512], label='Electrode A', color=PRIMARY_BLUE, linewidth=1.5)
ax.plot(t_vc[:512], sig2_vc[:512], label='Electrode B', color=PRIMARY_RED, linewidth=1.5, alpha=0.7)
ax.set_title('Volume Conduction Scenario\n(Same source → two electrodes)', 
             fontsize=12, fontweight='bold', color=PRIMARY_RED)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.legend()
ax.set_xlim(0, 2)

# Top Right: True Connectivity - Time series
ax = axes[0, 1]
ax.plot(t_tc[:512], sig1_tc[:512], label='Signal A', color=PRIMARY_BLUE, linewidth=1.5)
ax.plot(t_tc[:512], sig2_tc[:512], label='Signal B', color=PRIMARY_RED, linewidth=1.5, alpha=0.7)
ax.set_title('True Connectivity Scenario\n(Consistent 45° phase lag)', 
             fontsize=12, fontweight='bold', color=PRIMARY_GREEN)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.legend()
ax.set_xlim(0, 2)

# Bottom: Bar comparison
ax = axes[1, 0]
metrics = ['PLV', 'PLI']
vc_values = [plv_vc, pli_vc]
x = np.arange(len(metrics))
width = 0.35

bars = ax.bar(x, vc_values, width, color=[PRIMARY_RED, PRIMARY_RED], alpha=0.8)
ax.set_ylabel('Metric Value', fontsize=11)
ax.set_title('Volume Conduction: Metric Values', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics, fontsize=12, fontweight='bold')
ax.set_ylim(0, 1.1)
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)

# Add value labels
for bar, val in zip(bars, vc_values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.03, 
            f'{val:.2f}', ha='center', fontsize=14, fontweight='bold')

# Annotations
ax.annotate('High! (FALSE positive)', xy=(0, plv_vc), xytext=(0.3, plv_vc + 0.15),
            fontsize=10, color=PRIMARY_RED, fontweight='bold',
            arrowprops=dict(arrowstyle='->', color=PRIMARY_RED))
ax.annotate('Low ✓ (Correct!)', xy=(1, pli_vc), xytext=(1.2, pli_vc + 0.25),
            fontsize=10, color=PRIMARY_GREEN, fontweight='bold',
            arrowprops=dict(arrowstyle='->', color=PRIMARY_GREEN))

ax = axes[1, 1]
tc_values = [plv_tc, pli_tc]

bars = ax.bar(x, tc_values, width, color=[PRIMARY_GREEN, PRIMARY_GREEN], alpha=0.8)
ax.set_ylabel('Metric Value', fontsize=11)
ax.set_title('True Connectivity: Metric Values', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics, fontsize=12, fontweight='bold')
ax.set_ylim(0, 1.1)
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)

# Add value labels
for bar, val in zip(bars, tc_values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.03,
            f'{val:.2f}', ha='center', fontsize=14, fontweight='bold')

# Annotations  
ax.annotate('High ✓', xy=(0, plv_tc), xytext=(0.3, plv_tc + 0.1),
            fontsize=10, color=PRIMARY_GREEN, fontweight='bold',
            arrowprops=dict(arrowstyle='->', color=PRIMARY_GREEN))
ax.annotate('High ✓', xy=(1, pli_tc), xytext=(1.2, pli_tc + 0.1),
            fontsize=10, color=PRIMARY_GREEN, fontweight='bold',
            arrowprops=dict(arrowstyle='->', color=PRIMARY_GREEN))

plt.tight_layout()
plt.show()

print("=" * 70)
print("KEY TAKEAWAY:")
print("=" * 70)
print(f"• Volume Conduction: PLV = {plv_vc:.2f} (HIGH - false positive!), PLI = {pli_vc:.2f} (LOW - correct!)")
print(f"• True Connectivity:  PLV = {plv_tc:.2f} (HIGH - correct!), PLI = {pli_tc:.2f} (HIGH - correct!)")
print()
print("PLI correctly rejects spurious connectivity while preserving true connectivity!")

---

## 11. Spatial Filtering: The Laplacian Reference

While robust metrics like PLI address volume conduction at the **analysis stage**, spatial filtering addresses it at the **preprocessing stage**.

### The Surface Laplacian

The **Surface Laplacian** (or Current Source Density) transforms EEG data to emphasize **local** sources and attenuate **distant** ones.

**Intuition**: Instead of measuring voltage at each electrode, we measure how different that electrode is from its neighbors.

$$\nabla^2 V(x, y) = \frac{\partial^2 V}{\partial x^2} + \frac{\partial^2 V}{\partial y^2}$$

### How It Helps

| Property | Raw EEG | After Laplacian |
|----------|---------|-----------------|
| Spatial resolution | Low (blurred) | Higher (sharper) |
| Volume conduction | Severe | Reduced |
| Distant sources | Visible | Attenuated |
| Local sources | Mixed with neighbors | Enhanced |

### Practical Considerations

**Pros:**
- ✅ Reduces volume conduction before connectivity analysis
- ✅ Can be combined with any connectivity metric
- ✅ Implemented in standard toolboxes (MNE, EEGLAB)

**Cons:**
- ❌ Requires accurate electrode positions
- ❌ Edge effects at boundary electrodes
- ❌ May reduce SNR if sources are deep

> 📝 **Note**: For hyperscanning, Laplacian is particularly useful because within-brain volume conduction is the main concern, not between-brain (which is impossible!).

In [ ]:
# ============================================================================
# VISUALIZATION 13: Laplacian Effect - Conceptual Illustration
# ============================================================================

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Create a simple 2D "scalp" with electrode positions
n_grid = 50
x = np.linspace(-1, 1, n_grid)
y = np.linspace(-1, 1, n_grid)
X, Y = np.meshgrid(x, y)

# Create head mask (circular)
head_mask = X**2 + Y**2 <= 1

# Panel 1: Original source (diffuse)
ax = axes[0]

# Single source that spreads
source_x, source_y = -0.2, 0.3
sigma = 0.4  # Spread due to volume conduction
voltage = np.exp(-((X - source_x)**2 + (Y - source_y)**2) / (2 * sigma**2))
voltage[~head_mask] = np.nan

im = ax.imshow(voltage, extent=[-1, 1, -1, 1], cmap='RdBu_r', 
               vmin=-1, vmax=1, origin='lower')
ax.contour(X, Y, head_mask.astype(float), levels=[0.5], colors='black', linewidths=2)
ax.set_title('Raw EEG Voltage\n(Volume Conduction Spread)', fontsize=12, fontweight='bold')
ax.set_xlabel('Left ← → Right')
ax.set_ylabel('Posterior ← → Anterior')
ax.plot(source_x, source_y, 'k*', markersize=15, label='True source')
ax.legend(loc='lower right')
ax.set_aspect('equal')

# Panel 2: After Laplacian (more focal)
ax = axes[1]

sigma_lap = 0.15  # Much more focal after Laplacian
voltage_lap = np.exp(-((X - source_x)**2 + (Y - source_y)**2) / (2 * sigma_lap**2))
# Laplacian creates negative surround
voltage_lap = voltage_lap - 0.3 * np.exp(-((X - source_x)**2 + (Y - source_y)**2) / (2 * 0.4**2))
voltage_lap[~head_mask] = np.nan

im2 = ax.imshow(voltage_lap, extent=[-1, 1, -1, 1], cmap='RdBu_r',
                vmin=-0.5, vmax=1, origin='lower')
ax.contour(X, Y, head_mask.astype(float), levels=[0.5], colors='black', linewidths=2)
ax.set_title('After Surface Laplacian\n(More Focal)', fontsize=12, fontweight='bold')
ax.set_xlabel('Left ← → Right')
ax.set_ylabel('Posterior ← → Anterior')
ax.plot(source_x, source_y, 'k*', markersize=15, label='True source')
ax.legend(loc='lower right')
ax.set_aspect('equal')

# Panel 3: Profile comparison
ax = axes[2]

# Extract horizontal profile through source
y_idx = int((source_y + 1) / 2 * n_grid)
profile_raw = np.exp(-((x - source_x)**2) / (2 * 0.4**2))
profile_lap = np.exp(-((x - source_x)**2) / (2 * 0.15**2)) - \
              0.3 * np.exp(-((x - source_x)**2) / (2 * 0.4**2))

ax.plot(x, profile_raw, linewidth=3, label='Raw EEG', color=PRIMARY_RED)
ax.plot(x, profile_lap, linewidth=3, label='After Laplacian', color=PRIMARY_BLUE)
ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax.axvline(x=source_x, color='gray', linestyle=':', alpha=0.7)

ax.set_xlabel('Position (normalized)', fontsize=11)
ax.set_ylabel('Amplitude', fontsize=11)
ax.set_title('Horizontal Profile Through Source', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.set_xlim(-1, 1)

# Add annotation
ax.annotate('Wide spread\n(neighbors affected)', xy=(0.3, 0.6), 
            xytext=(0.6, 0.8), fontsize=9, color=PRIMARY_RED,
            arrowprops=dict(arrowstyle='->', color=PRIMARY_RED))
ax.annotate('Focal peak\n(local only)', xy=(source_x, 1), 
            xytext=(source_x - 0.5, 0.7), fontsize=9, color=PRIMARY_BLUE,
            arrowprops=dict(arrowstyle='->', color=PRIMARY_BLUE))

plt.tight_layout()
plt.show()

print("The Laplacian transform sharpens spatial resolution,")
print("making nearby electrodes more independent and reducing spurious connectivity.")

---

## 12. Hands-On Exercises

Now it's your turn to explore volume conduction and robust connectivity metrics!

In [ ]:
# ============================================================================
# EXERCISE 1: Varying Mixing Strength
# ============================================================================
# 
# Explore how PLV and PLI change as volume conduction strength increases.
#
# TODO:
# 1. Create a range of mixing strengths from 0.1 to 0.9
# 2. For each strength, simulate volume conduction and compute PLV and PLI
# 3. Plot both metrics as a function of mixing strength
# 4. At what mixing strength does PLV become misleadingly high?
#
# Expected outcome: PLV should increase with mixing, PLI should stay low
# ============================================================================

# Your code here:
mixing_strengths = np.linspace(0.1, 0.9, 9)
plv_values = []
pli_values = []

for strength in mixing_strengths:
    t, s1, s2 = simulate_volume_conduction_scenario(
        fs=fs, duration=5.0, mixing_strength=strength, seed=42
    )
    # Compute PLV
    a1 = scipy.signal.hilbert(s1)
    a2 = scipy.signal.hilbert(s2)
    phase_diff = np.angle(a2) - np.angle(a1)
    plv = np.abs(np.mean(np.exp(1j * phase_diff)))
    plv_values.append(plv)
    
    # Compute PLI
    pli = compute_pli(s1, s2)
    pli_values.append(pli)

# Plot
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(mixing_strengths, plv_values, 'o-', linewidth=2, markersize=8, 
        color=PRIMARY_RED, label='PLV (vulnerable)')
ax.plot(mixing_strengths, pli_values, 's-', linewidth=2, markersize=8,
        color=PRIMARY_BLUE, label='PLI (robust)')
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Threshold')
ax.set_xlabel('Volume Conduction Strength', fontsize=12)
ax.set_ylabel('Metric Value', fontsize=12)
ax.set_title('Exercise 1: PLV vs PLI Under Increasing Volume Conduction', 
             fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.set_ylim(0, 1.05)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Observation: PLV increases with mixing strength (false positives!)")
print("PLI remains low regardless of mixing strength (correct behavior)")

In [ ]:
# ============================================================================
# EXERCISE 2: Effect of Phase Lag on PLI
# ============================================================================
#
# Explore how different phase lags affect PLI detection.
#
# TODO:
# 1. Create true connectivity scenarios with phase lags from 0 to π
# 2. Compute PLI for each phase lag
# 3. At which phase lags is PLI highest? Lowest?
# 4. Why does PLI = 0 at phase lag = 0 and π?
# ============================================================================

# Your code here:
phase_lags = np.linspace(0, np.pi, 19)
pli_by_lag = []

for lag in phase_lags:
    t, s1, s2 = simulate_true_connectivity_scenario(
        fs=fs, duration=5.0, phase_lag=lag, noise_level=0.1, seed=42
    )
    pli = compute_pli(s1, s2)
    pli_by_lag.append(pli)

# Plot
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(np.degrees(phase_lags), pli_by_lag, 'o-', linewidth=2, markersize=8,
        color=PRIMARY_GREEN)
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=90, color=SECONDARY_ORANGE, linestyle=':', alpha=0.7, 
           label='90° (maximum sensitivity)')
ax.set_xlabel('Phase Lag (degrees)', fontsize=12)
ax.set_ylabel('PLI Value', fontsize=12)
ax.set_title('Exercise 2: PLI Sensitivity to Phase Lag', fontsize=13, fontweight='bold')
ax.set_xticks([0, 45, 90, 135, 180])
ax.legend(fontsize=11)
ax.set_ylim(0, 1.05)
ax.grid(True, alpha=0.3)

# Annotations
ax.annotate('0° = Volume conduction zone\n(PLI blind here)', 
            xy=(0, pli_by_lag[0]), xytext=(30, 0.3),
            fontsize=10, arrowprops=dict(arrowstyle='->', color='black'))
ax.annotate('180° = Anti-phase\n(PLI also blind)', 
            xy=(180, pli_by_lag[-1]), xytext=(140, 0.3),
            fontsize=10, arrowprops=dict(arrowstyle='->', color='black'))

plt.tight_layout()
plt.show()

print("Key insight: PLI is most sensitive around 90° phase lag")
print("It's blind to 0° (volume conduction) and 180° (anti-phase) relationships")

In [ ]:
# ============================================================================
# EXERCISE 3: Mixed Scenario - True Connectivity + Volume Conduction
# ============================================================================
#
# In reality, we often have BOTH true connectivity AND volume conduction.
# Let's explore how PLI handles this challenging scenario.
#
# TODO:
# 1. Create signals with true phase-lagged connectivity
# 2. Add varying amounts of volume conduction contamination
# 3. Compare PLV and PLI as contamination increases
# ============================================================================

# Create base signals with true connectivity (45° phase lag)
np.random.seed(42)
duration = 5.0
t = np.arange(0, duration, 1/fs)
freq = 10.0
noise_level = 0.1

# True sources with phase relationship
source_A = np.sin(2 * np.pi * freq * t)
source_B = np.sin(2 * np.pi * freq * t + np.pi/4)  # 45° ahead

# Common volume conduction source
common_source = np.sin(2 * np.pi * freq * t + np.pi/3)  # Different phase

# Vary the contamination level
contamination_levels = np.linspace(0, 1, 11)
plv_mixed = []
pli_mixed = []

for contam in contamination_levels:
    # Mix true signal with volume conduction contamination
    signal_A = (1 - contam) * source_A + contam * common_source + noise_level * np.random.randn(len(t))
    signal_B = (1 - contam) * source_B + contam * common_source + noise_level * np.random.randn(len(t))
    
    # Compute metrics
    a1 = scipy.signal.hilbert(signal_A)
    a2 = scipy.signal.hilbert(signal_B)
    phase_diff = np.angle(a2) - np.angle(a1)
    plv = np.abs(np.mean(np.exp(1j * phase_diff)))
    pli = compute_pli(signal_A, signal_B)
    
    plv_mixed.append(plv)
    pli_mixed.append(pli)

# Plot
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(contamination_levels * 100, plv_mixed, 'o-', linewidth=2, markersize=8,
        color=PRIMARY_RED, label='PLV')
ax.plot(contamination_levels * 100, pli_mixed, 's-', linewidth=2, markersize=8,
        color=PRIMARY_BLUE, label='PLI')

# Reference lines
ax.axhline(y=plv_mixed[0], color=PRIMARY_RED, linestyle=':', alpha=0.5)
ax.axhline(y=pli_mixed[0], color=PRIMARY_BLUE, linestyle=':', alpha=0.5)

ax.set_xlabel('Volume Conduction Contamination (%)', fontsize=12)
ax.set_ylabel('Metric Value', fontsize=12)
ax.set_title('Exercise 3: True Connectivity Masked by Volume Conduction', 
             fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.set_ylim(0, 1.05)
ax.grid(True, alpha=0.3)

# Annotations
ax.fill_between([0, 30], [0, 0], [1.1, 1.1], color=PRIMARY_GREEN, alpha=0.1, 
                label='Low contamination')
ax.fill_between([70, 100], [0, 0], [1.1, 1.1], color=PRIMARY_RED, alpha=0.1,
                label='High contamination')

plt.tight_layout()
plt.show()

print("Observation: As volume conduction contamination increases:")
print("• PLV becomes inflated (contamination adds to the signal)")
print("• PLI decreases as the true phase relationship is masked")

In [ ]:
# ============================================================================
# EXERCISE 4: Electrode Distance and Volume Conduction
# ============================================================================
#
# Volume conduction effects are stronger for nearby electrodes.
# Let's quantify this relationship.
#
# TODO:
# 1. Simulate electrodes at varying distances from a source
# 2. Compute pairwise PLV between a reference electrode and all others
# 3. How does spurious connectivity relate to electrode distance?
# ============================================================================

# Simulate a head with one source and electrodes at varying distances
np.random.seed(42)
source_position = np.array([0.0, 0.3])  # Source location

# Create electrodes in a line from close to far
n_test_electrodes = 8
electrode_x = np.linspace(-0.1, 0.8, n_test_electrodes)
electrode_y = np.ones(n_test_electrodes) * 0.3

# Reference electrode (closest to source)
ref_x, ref_y = -0.1, 0.3

# Compute distances from source
distances = np.sqrt((electrode_x - source_position[0])**2 + 
                    (electrode_y - source_position[1])**2)
ref_distance = np.sqrt((ref_x - source_position[0])**2 + 
                        (ref_y - source_position[1])**2)

# Generate source signal
t = np.arange(0, 2, 1/fs)
source = np.sin(2 * np.pi * 10 * t)

# Generate electrode signals with distance-based attenuation
noise_level = 0.2
ref_signal = source / (ref_distance + 0.1) + noise_level * np.random.randn(len(t))

plv_by_distance = []
for i in range(n_test_electrodes):
    weight = 1 / (distances[i] + 0.1)  # Inverse distance weighting
    electrode_signal = source * weight + noise_level * np.random.randn(len(t))
    
    # Compute PLV with reference
    a1 = scipy.signal.hilbert(ref_signal)
    a2 = scipy.signal.hilbert(electrode_signal)
    phase_diff = np.angle(a2) - np.angle(a1)
    plv = np.abs(np.mean(np.exp(1j * phase_diff)))
    plv_by_distance.append(plv)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Electrode layout
ax = axes[0]
ax.scatter([source_position[0]], [source_position[1]], s=200, c='gold', 
           marker='*', edgecolor='black', linewidths=2, label='Source', zorder=10)
ax.scatter([ref_x], [ref_y], s=150, c=PRIMARY_BLUE, marker='o', 
           edgecolor='black', linewidths=1, label='Reference electrode')
ax.scatter(electrode_x, electrode_y, s=100, c=distances, cmap='coolwarm',
           edgecolor='black', linewidths=1)

# Draw distance lines
for i in range(n_test_electrodes):
    ax.plot([source_position[0], electrode_x[i]], 
            [source_position[1], electrode_y[i]], 
            'k--', alpha=0.3, linewidth=1)

ax.set_xlim(-0.3, 1)
ax.set_ylim(-0.1, 0.7)
ax.set_xlabel('X position', fontsize=11)
ax.set_ylabel('Y position', fontsize=11)
ax.set_title('Electrode Layout', fontsize=12, fontweight='bold')
ax.legend(loc='upper right')
ax.set_aspect('equal')

# Right: PLV vs distance
ax = axes[1]
ax.plot(distances, plv_by_distance, 'o-', linewidth=2, markersize=10, color=PRIMARY_RED)
ax.set_xlabel('Distance from Source', fontsize=12)
ax.set_ylabel('PLV with Reference Electrode', fontsize=12)
ax.set_title('Spurious PLV Decreases with Distance', fontsize=12, fontweight='bold')
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Typical threshold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insight: Nearby electrodes show high spurious connectivity due to volume conduction.")
print("This is why within-brain connectivity must be interpreted with caution!")

In [ ]:
# ============================================================================
# EXERCISE 5: Frequency Band Effects
# ============================================================================
#
# Volume conduction affects all frequencies, but SNR varies by band.
# Let's explore how different frequency bands are affected.
#
# TODO:
# 1. Create volume conduction scenario with broadband source
# 2. Filter into different frequency bands
# 3. Compare PLV across bands
# ============================================================================

from scipy.signal import butter, filtfilt

def bandpass_filter(data, lowcut, highcut, fs, order=4):
    """Apply bandpass filter to data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

# Create volume conduction scenario with multiple frequency components
np.random.seed(42)
t = np.arange(0, 10, 1/fs)

# Source with multiple frequency components
source_multi = (np.sin(2 * np.pi * 4 * t) +      # Theta
                0.8 * np.sin(2 * np.pi * 10 * t) + # Alpha
                0.5 * np.sin(2 * np.pi * 20 * t) + # Beta
                0.3 * np.sin(2 * np.pi * 35 * t))  # Gamma

# Volume conduction to two electrodes
noise_level = 0.3
elec_1 = source_multi + noise_level * np.random.randn(len(t))
elec_2 = 0.7 * source_multi + noise_level * np.random.randn(len(t))

# Define frequency bands
bands = {
    'Theta (4-8 Hz)': (4, 8),
    'Alpha (8-13 Hz)': (8, 13),
    'Beta (13-30 Hz)': (13, 30),
    'Gamma (30-45 Hz)': (30, 45)
}

plv_by_band = {}
pli_by_band = {}

for band_name, (low, high) in bands.items():
    # Filter both signals
    filtered_1 = bandpass_filter(elec_1, low, high, fs)
    filtered_2 = bandpass_filter(elec_2, low, high, fs)
    
    # Compute PLV
    a1 = scipy.signal.hilbert(filtered_1)
    a2 = scipy.signal.hilbert(filtered_2)
    phase_diff = np.angle(a2) - np.angle(a1)
    plv = np.abs(np.mean(np.exp(1j * phase_diff)))
    pli = compute_pli(filtered_1, filtered_2)
    
    plv_by_band[band_name] = plv
    pli_by_band[band_name] = pli

# Plot
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(bands))
width = 0.35

bars1 = ax.bar(x - width/2, list(plv_by_band.values()), width, 
               label='PLV (vulnerable)', color=PRIMARY_RED, alpha=0.8)
bars2 = ax.bar(x + width/2, list(pli_by_band.values()), width,
               label='PLI (robust)', color=PRIMARY_BLUE, alpha=0.8)

ax.set_ylabel('Metric Value', fontsize=12)
ax.set_xlabel('Frequency Band', fontsize=12)
ax.set_title('Exercise 5: Volume Conduction Effects Across Frequency Bands', 
             fontsize=13, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(list(bands.keys()), fontsize=10)
ax.legend(fontsize=11)
ax.set_ylim(0, 1.1)
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{height:.2f}', ha='center', va='bottom', fontsize=9)
for bar in bars2:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{height:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("Observation: PLV shows high values across ALL bands due to volume conduction.")
print("PLI remains low, correctly indicating no true phase-lagged connectivity.")

---

## 13. Summary

### What We Learned

| Concept | Key Insight |
|---------|-------------|
| **Volume Conduction** | Brain activity spreads instantaneously through tissue, causing multiple electrodes to record the same source |
| **The Problem** | This creates **spurious connectivity** that is NOT due to neural communication |
| **Zero-Lag Signature** | Volume conduction produces **zero time lag** and **zero phase difference** |
| **PLV Vulnerability** | Standard PLV cannot distinguish true from spurious connectivity |
| **PLI Robustness** | Phase Lag Index ignores zero-lag relationships, making it robust to volume conduction |
| **Imaginary Coherence** | Uses only the imaginary part of coherence, which is zero for volume conduction |
| **Spatial Filtering** | Laplacian transform reduces spatial spread before connectivity analysis |

### Practical Guidelines

1. **Always consider volume conduction** when analyzing EEG connectivity
2. **Use robust metrics** (PLI, wPLI, ImCoh) for within-brain connectivity
3. **Distant electrode pairs** are less affected than nearby pairs
4. **Hyperscanning advantage**: Between-brain connectivity is immune to volume conduction!
5. **Combine approaches**: Use both robust metrics AND spatial filtering for best results

### Coming Up Next

In the following notebooks, we'll implement each of these robust metrics in detail:
- **F01**: Spectral Coherence (and its imaginary part)
- **G01**: Phase Locking Value (the vulnerable baseline)
- **G02**: Phase Lag Index (robust alternative)
- **G03**: Weighted Phase Lag Index (even more robust)

---

## 14. Discussion Questions

1. **Why is volume conduction particularly problematic for EEG compared to other neuroimaging methods like fMRI?**
   
   *Hint: Think about the physical principles of signal propagation and spatial resolution.*

2. **In hyperscanning studies, why do we say that between-brain connectivity is "immune" to volume conduction?**
   
   *Hint: Consider the physical distance and the nature of electromagnetic field propagation.*

3. **PLI is blind to 0° and 180° phase relationships. Could there be TRUE neural connectivity with exactly 0° phase lag? How would we detect it?**
   
   *Hint: Think about what could cause true zero-lag connectivity and whether we can distinguish it from volume conduction.*

4. **If you were designing a hyperscanning experiment, would you be more concerned about volume conduction for within-brain or between-brain connectivity? Why?**
   
   *Hint: Consider where volume conduction CAN and CANNOT occur.*

5. **Some researchers argue that using only robust metrics like PLI might cause us to MISS real connectivity. Do you agree? What's the trade-off?**
   
   *Hint: Think about sensitivity vs. specificity in statistics.*

---

**Notebook completed!** 🎉

You now understand why volume conduction is the central problem in EEG connectivity analysis and how to address it using robust metrics like PLI. This knowledge is essential for all subsequent connectivity notebooks in this workshop.